In [1]:
import numpy as np
import copy
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import regularizers
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from abc import ABC, abstractmethod
from random import random
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import os
#initialize for random seeds/states
tf.keras.backend.clear_session()
os.environ['PYTHONHASHSEED'] = '0'
rng = np.random.default_rng(12345)
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
class paper_game(ABC):
    def __init__(self, start_state):
        self.state=start_state
    
    @abstractmethod
    def transition(state, action, player):
        #return next state
        pass
    
    @abstractmethod
    def reward(self, state, action, player):
        #return reward immediately after action
        pass
    
    @abstractmethod
    def policy(self, state):
        #return action
        pass
    
    @abstractmethod
    def Q_func(self, state, action):
        pass
    
    @abstractmethod
    def Q_update(self, state, action, reward_1, maxQ, player):
        pass

In [5]:
class tit_tac_toe(paper_game):
#state: 3x3 array to represent board config, with 1 representing self-checker, -1 representing opponent-checker, 0 representing empty
    def __init__(self, start_state=np.zeros((3,3), dtype=np.int8), epsilon=0.9, alpha=0.5, gamma=0.8):
        self.state=start_state
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.Q = np.zeros((19683, 9))  #initialize Q matrix with state 'reference' (see ref function) and action key (see key function)    
    
    def transition(self, state, action, player):
        #action: a list of row and column index to indicate which cell is chosen
        #player: 1 or -1
        #applicable to both tic_tac_toe and Connect-4
        #return next state
        new_state = copy.deepcopy(state)
        new_state[action[0]][action[1]] = player
        return new_state
    
    def win_status(self, s):
        #s: state, 
        #return: whether win or not
        win = 3 in np.sum(s, axis=0) or 3 in np.sum(s, axis=1) or np.sum(s.diagonal())==3 or np.sum(np.fliplr(s).diagonal())==3
        return win
    
    def ref(self, state):
        #change state matrix into vector
        ref = np.sum(np.matrix([1,3,3**2,3**3,3**4,3**5,3**6,3**7,3**8]) @ np.reshape((state + 1).flatten(),(9,1)))
        return ref
    
    def key(self, action):
        #change actions [r, c] into numbers 0-8
        return 3*action[0]+action[1]
    
    def action_list(self, state):
        act_list = np.where(state==0)
        return act_list

    def reward(self, state, action, player):
        s = player * self.transition(state, action, player)
        if self.win_status(s):
            reward = 100
        else:
            reward = 0
        return reward
    
    def Q_func(self, state, action):
        Q = self.Q[self.ref(state)][self.key(action)]
        return Q

    def Q_update(self, state, action, reward_1, maxQ, player):
        #state of board instead of current player's perspective
        #reward_1: reward of player 1 regardless of who is current player
        #maxQ: player's maxQ after another action
        #player: 1 or -1
        s = player*state
        self.Q[self.ref(s)][self.key(action)] += self.alpha*(reward_1*player + self.gamma*maxQ - self.Q[self.ref(s)][self.key(action)] )


    def random_move(self, state):
        act_list = self.action_list(state)
        i = np.random.randint(len(act_list[0]))
        action = [act_list[0][i],act_list[1][i]]
        return action

    def best_move(self, state):
        act_list = self.action_list(state)
        maxQ = -10000
        for i in range (len(act_list[0])):
            a = [act_list[0][i],act_list[1][i]]
            temp = self.Q_func(state, a)
            if temp > maxQ:
                maxQ = temp
                action = a
        return action, maxQ
    
    def policy(self, state):
        exploit = (not self.train) or rng.random()> self.epsilon
        if exploit:
            action, maxQ = self.best_move(state)
        else:
            action = self.random_move(state)
        if self.epsilon < 0.5:
            self.epsilon *= 0.9999
        else:
            self.epsilon *= 0.99999    
        return action
    
    def train(self, train_number=2000):
        self.train = True
        #loop by train_number
        for i in range (train_number):
            states =[]
            state = self.state #start state
            states.append(state)
            actions=[]
            r1=[0] #reward list at different times for player 1; for player -1: use zero sum property
            t = 0
            #choose who is X, i.e. plays first
            player = 1 #as this is self-play, we simply assume 1 always plays first and be playerX
            endgame = False
            while not endgame:
                #update his Q before action if previous has action
                s = player * state
                if t >=2:
                    action, maxQ = self.best_move(s)
                    self.Q_update(states[t-2], actions[t-2], r1[t], maxQ, player)
                    
                #action
                action = self.policy(s)
                actions.append(action)
                t += 1
                r1.append(player*self.reward(state, action, player))
                state=self.transition(state, action, player)
                states.append(state)
                #check win status and end game status, update Q for both players if end game
                endgame = self.win_status(s) or t==9
                if endgame:
                    self.Q_update(states[t-1], actions[t-1], r1[t], 0, player) #r1[t+1] is not used as there won't be t+1 when endgame is reached
                    self.Q_update(states[t-2], actions[t-2], r1[t], 0, -player)
                    
                #update player for next loop
                player *= -1
        self.last_episode = states
        
    def display_board(self, state):
        print('-------------------')
        print(state)
        print('-------------------')

    def display_episode(self):
        for i in range(len(self.last_episode)):
            print('state {} :'.format(i))
            self.display_board(self.last_episode[i])
            
    def play(self):
        self.train = False

In [8]:
test1 = tit_tac_toe()
test1.train(10000)
test1.display_episode()

state 0 :
-------------------
[[0 0 0]
 [0 0 0]
 [0 0 0]]
-------------------
state 1 :
-------------------
[[1 0 0]
 [0 0 0]
 [0 0 0]]
-------------------
state 2 :
-------------------
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0  0]]
-------------------
state 3 :
-------------------
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
-------------------
state 4 :
-------------------
[[ 1  0  0]
 [-1 -1  0]
 [ 0  0  1]]
-------------------
state 5 :
-------------------
[[ 1  0  0]
 [-1 -1  1]
 [ 0  0  1]]
-------------------
state 6 :
-------------------
[[ 1  0 -1]
 [-1 -1  1]
 [ 0  0  1]]
-------------------
state 7 :
-------------------
[[ 1  0 -1]
 [-1 -1  1]
 [ 1  0  1]]
-------------------
state 8 :
-------------------
[[ 1  0 -1]
 [-1 -1  1]
 [ 1 -1  1]]
-------------------
state 9 :
-------------------
[[ 1  1 -1]
 [-1 -1  1]
 [ 1 -1  1]]
-------------------
